In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [3]:
df = pd.read_csv("02_Test_Data_Set/mobiles.csv")
df.head(2)

,screen_size,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales
0,Very Small,64,2,1,1,1800,4.5,38645,32999,0.17,127.52
1,Small,64,4,2,1,2815,4.5,244,57149,0.04,1.39


### Q1.

In [6]:
stat_mean = df["sales"].mean()
stat_std  = df["sales"].std()
stat_out  = stat_mean + stat_std * 2
stat_out

146.55150129273218

In [7]:
df_q1 = df.loc[df["sales"] > stat_out, ].reset_index(drop = True)
df_q1.head(2)

,screen_size,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales
0,Medium,128,6,2,1,4000,4.6,122001,18999,0.09,231.79
1,Large,128,6,4,2,4500,4.5,267028,15999,0.20,427.22


In [8]:
len(df), len(df_q1)

(430, 16)

In [11]:
df_q1["idx"] = (df_q1["ROM"] / 32) + (df_q1["RAM"] / 2) + \
(df_q1["num_front_camera"] + df_q1["num_rear_camera"]) + \
(df_q1["battery_capacity"] / 1000)

In [12]:
df_q1.head(2)

,screen_size,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales,idx
0,Medium,128,6,2,1,4000,4.6,122001,18999,0.09,231.79,14.0
1,Large,128,6,4,2,4500,4.5,267028,15999,0.20,427.22,17.5


In [13]:
round(df_q1["idx"].mean(), 2)

11.01

### Q2.

In [16]:
df_q2 = df.loc[df["num_rear_camera"] != 1, "battery_capacity":]

In [17]:
df_q2.head(2)

,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales
1,2815,4.5,244,57149,0.04,1.39
4,2815,4.6,745,69149,0.02,5.15


In [23]:
round(df_q2.corr()["sales"][:-1].abs().max(), 2)

0.95

In [25]:
df_q2.corr()["sales"][:-1].abs().round(2).max()

0.95

### Q3

In [27]:
df_q3 = df.set_index("sales").reset_index()
df_q3.head(2)

,sales,screen_size,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent
0,127.52,Very Small,64,2,1,1,1800,4.5,38645,32999,0.17
1,1.39,Small,64,4,2,1,2815,4.5,244,57149,0.04


In [34]:
# df_q3_dum = pd.get_dummies(df_q3, columns = ["screen_size", "num_rear_camera"])
# df_q3_dum.head(2)

In [35]:
df_q3_dum = pd.get_dummies(df_q3, columns = ["screen_size"])
df_q3_dum.head(2)

,sales,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
0,127.52,64,2,1,1,1800,4.5,38645,32999,0.17,0,0,0,0,1
1,1.39,64,4,2,1,2815,4.5,244,57149,0.04,0,0,1,0,0


In [31]:
df_q3_dum.columns # 변수명에 띄어쓰기가 있는 경우 모델링시 문제 발생 가능.

Index(['sales', 'ROM', 'RAM', 'num_rear_camera', 'num_front_camera',
       'battery_capacity', 'ratings', 'num_of_ratings', 'sales_price',
       'discount_percent', 'screen_size_Large', 'screen_size_Medium',
       'screen_size_Small', 'screen_size_Very Large',
       'screen_size_Very Small'],
      dtype='object')

In [36]:
df_q3_dum.columns.str.replace(" ", "_")

Index(['sales', 'ROM', 'RAM', 'num_rear_camera', 'num_front_camera',
       'battery_capacity', 'ratings', 'num_of_ratings', 'sales_price',
       'discount_percent', 'screen_size_Large', 'screen_size_Medium',
       'screen_size_Small', 'screen_size_Very_Large',
       'screen_size_Very_Small'],
      dtype='object')

In [37]:
df_q3_dum.shape

(430, 15)

In [38]:
df_train, df_test = train_test_split(df_q3_dum, train_size = 0.8, random_state = 123)

In [41]:
model_nor = MinMaxScaler()
model_nor.fit(df_train)
arr_train_nor = model_nor.transform(df_train)
arr_test_nor  = model_nor.transform(df_test)

df_train_nor = pd.DataFrame(arr_train_nor, columns = df_train.columns)
df_test_nor  = pd.DataFrame(arr_test_nor,  columns = df_train.columns)

In [42]:
df_train_nor.head(2)

,sales,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
0,0.003948,0.047619,0.181818,0.333333,0.0,0.423077,0.625,0.003963,0.026073,0.093023,0.0,0.0,1.0,0.0,0.0
1,0.052148,0.238095,0.181818,0.000000,0.0,0.219615,1.000,0.011387,0.273927,0.186047,0.0,1.0,0.0,0.0,0.0


In [50]:
k = 3

model_knn = KNeighborsRegressor(n_neighbors = k)
model_knn.fit(X = arr_train_nor[:, 1:],
              y = arr_test_nor[:, 0])
pred = model_knn.predict(arr_test_nor[:, 1:])
mean_squared_error(y_true = arr_test_nor[:, 0], y_pred = pred) ** 0.5

0.08186677375964535

In [46]:
k = 3

model_knn = KNeighborsRegressor(n_neighbors = k)
model_knn.fit(X = df_train_nor.drop("sales", axis = 1),
              y = df_train_nor["sales"])
pred = model_knn.predict(df_test_nor.drop("sales", axis = 1))
mean_squared_error(y_true = df_test_nor["sales"], y_pred = pred) ** 0.5

0.08186677375964535

In [47]:
ls_rmse = []

for k in [3, 5, 7, 9, 11]:
    model_knn = KNeighborsRegressor(n_neighbors = k)
    model_knn.fit(X = df_train_nor.drop("sales", axis = 1),
                  y = df_train_nor["sales"])
    pred = model_knn.predict(df_test_nor.drop("sales", axis = 1))
    val_rmse = mean_squared_error(y_true = df_test_nor["sales"], y_pred = pred) ** 0.5
    ls_rmse = ls_rmse + [val_rmse]

In [48]:
ls_rmse

[0.08186677375964535,
 0.09879109824384892,
 0.107669855645971,
 0.11232111394853059,
 0.1136902366621185]

In [56]:
ser_method = pd.Series([method for method in dir(pd.Series) if method[0] != "_"])
ser_method[ser_method.str.contains("^c")]

29               cat
30              clip
31           combine
32     combine_first
33           compare
34    convert_dtypes
35              copy
36              corr
37             count
38               cov
39            cummax
40            cummin
41           cumprod
42            cumsum
dtype: object